In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer


In [26]:

df = pd.read_csv('TelecomFactory_train_data.csv')
# файл xlsx долго читалась, так что перевел ее в csv

In [27]:
df.head()

,CLIENT_ID,FLD_1,FLD_2,FLD_3,FLD_4,FLD_5,FLD_6,FLD_7,FLD_8,FLD_9,...,FLD_398,FLD_399,FLD_400,FLD_401,FLD_402,FLD_403,FLD_404,target_1,target_2,target_3
0,9,0,F,4,NaN,34.0,3,0,500000.0,431000.0,...,NaN,NaN,NaN,1.0,1.0,11,8,1,0,0
1,11,0,F,3,NaN,47.0,6,0,120000.0,130000.0,...,NaN,NaN,NaN,10.0,NaN,2,0,1,0,0
2,12,0,F,1,NaN,56.0,3,0,100000.0,100000.0,...,NaN,NaN,NaN,11.0,11.0,3,2,1,0,0
3,14,0,F,3,NaN,56.0,6,0,500000.0,50000.0,...,NaN,NaN,NaN,2.0,2.0,13,7,1,0,0
4,15,0,F,2,NaN,49.0,5,0,100000.0,460000.0,...,NaN,NaN,NaN,7.0,7.0,1,1,1,0,0


In [29]:

for col in df.columns:
    try:
        df[col] = df[col].astype(float)
    except ValueError:
        print(col)


FLD_2
FLD_124
FLD_125


In [30]:
df = pd.get_dummies(df, columns=['FLD_2'], drop_first=True)
df = pd.get_dummies(df, columns=['FLD_124'], drop_first=True)
df = pd.get_dummies(df, columns=['FLD_125'], drop_first=True)

df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120920 entries, 0 to 120919
Columns: 467 entries, CLIENT_ID to FLD_125_Филиал Южная столица
dtypes: float64(405), uint8(62)
memory usage: 380.8 MB


In [31]:
for col in df.columns:
    mt_percent = df[col].isnull().sum() / len(df) * 100
    if mt_percent > 50:
        df = df.drop(col, axis=1)
    else:
        mean_value = df[col].mean()
        df[col] = df[col].fillna(mean_value)

In [32]:
df.isnull().sum()

CLIENT_ID                          0
FLD_1                              0
FLD_3                              0
FLD_5                              0
FLD_6                              0
                                  ..
FLD_125_Филиал Уральск             0
FLD_125_Филиал Усть-Каменогорск    0
FLD_125_Филиал Шымкент             0
FLD_125_Филиал Экибастуз           0
FLD_125_Филиал Южная столица       0
Length: 251, dtype: int64

In [34]:
for col in df.columns:
   mins = df[col].min()
   maxs = df[col].max()
   df[col] = (df[col] - mins) / (maxs - mins)



In [35]:
df.head()

,CLIENT_ID,FLD_1,FLD_3,FLD_5,FLD_6,FLD_7,FLD_8,FLD_9,FLD_10,FLD_11,...,FLD_125_Филиал Талгар,FLD_125_Филиал Талдыкорган,FLD_125_Филиал Тараз,FLD_125_Филиал Темиртау,FLD_125_Филиал Туркестан,FLD_125_Филиал Уральск,FLD_125_Филиал Усть-Каменогорск,FLD_125_Филиал Шымкент,FLD_125_Филиал Экибастуз,FLD_125_Филиал Южная столица
0,0.000000,NaN,0.833333,0.557292,0.500,0.0,2.776698e-06,0.000061,0.021363,0.009434,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000008,NaN,0.666667,0.625000,0.875,0.0,6.664075e-07,0.000018,0.008545,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000012,NaN,0.333333,0.671875,0.500,0.0,5.553396e-07,0.000014,0.005127,0.018868,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000019,NaN,0.666667,0.671875,0.875,0.0,2.776698e-06,0.000007,0.004273,0.094340,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000023,NaN,0.500000,0.635417,0.750,0.0,5.553396e-07,0.000065,0.001709,0.094340,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [36]:
df = df.drop('CLIENT_ID', axis=1)

In [42]:
data = df.copy()
data = data.to_numpy()
imputer = SimpleImputer(strategy='median')
data = imputer.fit_transform(data)
target_1_col_index = 4  # Replace 4 with the actual index of the 'target_1' column in your data
target_1_col = data[:, target_1_col_index]


In [46]:
data[:, target_1_col_index]


array([2.77669795e-06, 6.66407507e-07, 5.55339590e-07, ...,
       8.33009384e-07, 5.55339590e-07, 4.99805631e-07])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

targets = ['target_1', 'target_2', 'target_3']
for target in targets:
    y = data[target]
    X = data.drop([target], axis=1)
    pca = PCA()
    pca.fit(X)
    cumsum = np.cumsum(pca.explained_variance_ratio_)
    n_components = np.argmax(cumsum >= 0.95) + 1
    print("Оптимальное количество компонент для целевой переменной", target, ":", n_components)
    pca = PCA(n_components=n_components)
    X_reduced = pca.fit_transform(X)
    model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42)
    scores = cross_val_score(model, X_reduced, y, cv=5)
    print("Количество фичей для целевой переменной", target, ":", n_components)
    print("Средняя точность модели случайного леса для целевой переменной", target, ":", scores.mean())

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices